In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("clustering")

import os
from utils import get_data_dir, prinT

In [3]:
import pandas as pd
MAG_paper_df = pd.read_parquet(os.path.join(get_data_dir(), "MAG_paper.parquet"))
MAG_paper_df[MAG_paper_df['Year'] == 2010].to_parquet(os.path.join(get_data_dir(), '2010s', "MAG_paper.parquet"))

data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data


In [4]:
import pickle
MAG_paper_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s","MAG_paper.parquet"))
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s/clustering_results","cluster_df.parquet"))

data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data


In [5]:
MAG_paper_df

,Year,DocType,VenueID
PaperID,,,
2049933365,2010,Journal,80951755
2038148770,2010,Journal,119525064
2373199189,2010,Journal,2764482698
2289331308,2010,Journal,2764425571
2784227654,2010,Journal,77020770
...,...,...,...
2040927587,2010,Journal,135866459
3030134270,2010,Journal,2764740593
2087753748,2010,Journal,1567313


In [6]:
cluster_df

,Scopus_label,movMF_label,movMF_distance,x_val,y_val,kmeans_label,kmeans_distance,skm_label,skm_distance,spectral_label,n2v_kmeans_label,cm_kmeans_label,bert_kmeans_label,gnn_kmeans_label
202381698,Multidisciplinary,22,0.445886,-67.928200,15.572327,17,0.628846,20,0.444711,24,3,21,18,0
137773608,Multidisciplinary,22,0.590942,-68.405334,-55.633186,17,0.735654,20,0.559494,24,20,21,8,10
125754415,Multidisciplinary,22,0.574571,-68.448853,-55.613579,17,0.705024,20,0.550081,24,20,21,8,0
3880285,Multidisciplinary,23,0.618842,-68.407288,-55.634430,8,0.724859,17,0.610582,24,20,21,18,10
111155417,Chemistry,23,0.220853,-54.506985,-61.217068,11,0.495787,12,0.198758,14,4,24,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2764485818,Medicine,21,0.268070,-28.969574,34.819569,18,0.542531,14,0.251015,8,18,1,23,11
83454320,Arts and Humanities,25,0.034777,78.609909,31.736822,13,0.251599,3,0.056642,21,5,1,23,14
16507453,Arts and Humanities,6,0.113656,89.206772,17.625090,13,0.307089,3,0.128660,21,5,1,23,14
121509672,Agricultural and Biological Sciences,5,0.179111,-36.757057,-0.591017,21,0.430040,16,0.207435,10,5,1,23,1


# Generate `valid_paper.pkl`

In [7]:
# 获取所有 PaperID 和对应的期刊 ID
prinT("Mapping paper ID to venue ID")
paper_to_venue = MAG_paper_df["VenueID"].to_dict()
# 获取期刊 ID 和对应的 Scopus_label
prinT("Mapping venue ID to Scopus label")
venue_to_Scopus_label = cluster_df["Scopus_label"].to_dict()
# 获取期刊 ID 和对应的 movMF_label
prinT("Mapping venue ID to movMF label")
venue_to_movMF_label = cluster_df["movMF_label"].to_dict()
# 获取期刊 ID 和对应的 kmeans_label
prinT("Mapping venue ID to KMeans label")
venue_to_kmeans_label = cluster_df["kmeans_label"].to_dict()
# 获取期刊 ID 和对应的 skm_label
prinT("Mapping venue ID to SKM label")
venue_to_skm_label = cluster_df["skm_label"].to_dict()
# 获取期刊 ID 和对应的 spectral_label
prinT("Mapping venue ID to spectral label")
venue_to_spectral_label = cluster_df["spectral_label"].to_dict()
# 获取期刊 ID 和对应的 n2v_kmeans_label
prinT("Mapping venue ID to n2v_kmeans label")
venue_to_n2v_kmeans_label = cluster_df["n2v_kmeans_label"].to_dict()
# 获取期刊 ID 和对应的 cm_kmeans_label
prinT("Mapping venue ID to cm_kmeans label")
venue_to_cm_kmeans_label = cluster_df["cm_kmeans_label"].to_dict()
# 获取期刊 ID 和对应的 bert_kmeans_label
prinT("Mapping venue ID to bert_kmeans label")
venue_to_bert_kmeans_label = cluster_df["bert_kmeans_label"].to_dict()
# 获取期刊 ID 和对应的 gnn_kmeans_label
prinT("Mapping venue ID to gnn_kmeans label")
venue_to_gnn_kmeans_label = cluster_df["gnn_kmeans_label"].to_dict()

# 过滤出有效的 PaperID 和对应 Scopus_label
prinT("Filtering out invalid papers")
valid_papers = {'Scopus_label': {pid: venue_to_Scopus_label[vid] for pid, vid in paper_to_venue.items() if vid in 
                                 venue_to_Scopus_label},
                'movMF_label': {pid: venue_to_movMF_label[vid] for pid, vid in paper_to_venue.items() if vid in 
                                  venue_to_movMF_label},
                'kmeans_label': {pid: venue_to_kmeans_label[vid] for pid, vid in paper_to_venue.items() if vid in 
                                  venue_to_kmeans_label},
                'skm_label': {pid: venue_to_skm_label[vid] for pid, vid in paper_to_venue.items() if vid in 
                                  venue_to_skm_label},
                'spectral_label': {pid: venue_to_spectral_label[vid] for pid, vid in paper_to_venue.items() if vid in 
                                  venue_to_spectral_label},
                'n2v_kmeans_label': {pid: venue_to_n2v_kmeans_label[vid] for pid, vid in paper_to_venue.items() if vid in 
                                  venue_to_n2v_kmeans_label},
                'cm_kmeans_label': {pid: venue_to_cm_kmeans_label[vid] for pid, vid in paper_to_venue.items() if vid in 
                                  venue_to_cm_kmeans_label},
                'bert_kmeans_label': {pid: venue_to_bert_kmeans_label[vid] for pid, vid in paper_to_venue.items() if vid in
                                      venue_to_bert_kmeans_label},
                'gnn_kmeans_label': {pid: venue_to_gnn_kmeans_label[vid] for pid, vid in paper_to_venue.items() if vid in
                                        venue_to_gnn_kmeans_label},
                                  }

2025-02-11 17:09:34 -- Mapping paper ID to venue ID
2025-02-11 17:09:35 -- Mapping venue ID to Scopus label
2025-02-11 17:09:35 -- Mapping venue ID to movMF label
2025-02-11 17:09:35 -- Mapping venue ID to KMeans label
2025-02-11 17:09:35 -- Mapping venue ID to SKM label
2025-02-11 17:09:35 -- Mapping venue ID to spectral label
2025-02-11 17:09:35 -- Mapping venue ID to n2v_kmeans label
2025-02-11 17:09:35 -- Mapping venue ID to cm_kmeans label
2025-02-11 17:09:35 -- Mapping venue ID to bert_kmeans label
2025-02-11 17:09:35 -- Mapping venue ID to gnn_kmeans label
2025-02-11 17:09:35 -- Filtering out invalid papers


In [8]:
len(valid_papers['Scopus_label'])

1915871

In [9]:
with open(os.path.join(get_data_dir(), "2010s/classification_tasks","valid_papers.pkl"), "wb") as f:
    pickle.dump(valid_papers, f)

data_dir:  /home/lyuzhuoqi/projects/clustering/data


# Load `valid_paper.pkl`

In [10]:
import pickle
with open(os.path.join(get_data_dir(), "2010s/classification_tasks", "valid_papers.pkl"), "rb") as f:
    valid_papers = pickle.load(f)

data_dir:  /home/lyuzhuoqi/projects/clustering/data


# Construct `paper_abstract_labels.json`

In [10]:
import ijson
import json
from tqdm import tqdm
import os

def extract_valid_papers(json_path, valid_papers, output_path):
    """
    从平坦的 JSON 文件（{ "PaperID": "abstract", ... }）中提取指定的 PaperID 和摘要。
    
    Args:
        json_path (str): JSON 文件路径。
        valid_papers (dict): PaperID 和其对应标签的字典。
        output_path (str): 提取后保存的文件路径。
    
    Returns:
        int: 匹配到的 PaperID 数量。
    """
    matched_count = 0

    # 获取文件大小用于进度条
    file_size = os.path.getsize(json_path)

    with open(json_path, 'r', encoding='utf-8') as input_file, open(output_path, 'w', encoding='utf-8') as output_file:
        # 初始化 tqdm 进度条
        with tqdm(total=file_size, desc="Processing JSON", unit="B", unit_scale=True) as pbar:
            parser = ijson.kvitems(input_file, "")  # 解析顶层键值对
            batch = []
            batch_size = 50000  # 增大批量写入规模
            counter = 0  # 控制进度条更新频率

            for paper_id, abstract in parser:
                counter += 1
                if counter % 10000 == 0:  # 每 1000 条更新一次进度条
                    pbar.update(input_file.tell() - pbar.n)

                try:
                    paper_id = int(paper_id)
                    if paper_id in valid_papers['Scopus_label']:
                        result = {
                            "PaperID": paper_id,
                            "abstract": abstract,
                            "Scopus_label": valid_papers['Scopus_label'][paper_id],
                            "movMF_label": valid_papers['movMF_label'][paper_id],
                            "kmeans_label": valid_papers['kmeans_label'][paper_id],
                            "skm_label": valid_papers['skm_label'][paper_id],
                            "spectral_label": valid_papers['spectral_label'][paper_id],
                            "n2v_kmeans_label": valid_papers['n2v_kmeans_label'][paper_id],
                            "cm_kmeans_label": valid_papers['cm_kmeans_label'][paper_id],
                            "bert_kmeans_label": valid_papers['bert_kmeans_label'][paper_id],
                            "gnn_kmeans_label": valid_papers['gnn_kmeans_label'][paper_id],
                        }
                        batch.append(result)
                        matched_count += 1
                except ValueError:
                    continue

                if len(batch) >= batch_size:
                    output_file.write("\n".join(json.dumps(item) for item in batch) + "\n")
                    batch = []

            if batch:  # 写入剩余的批量数据
                output_file.write("\n".join(json.dumps(item) for item in batch) + "\n")

    prinT(f"Successfully extracted {matched_count} paper abstracts")
    return matched_count

In [11]:
large_json_path = os.path.join(get_data_dir(), "MAG_abstracts.json")
test_json_path = os.path.join(get_data_dir(), "MAG_abstracts_test.json")
output_path = os.path.join(get_data_dir(), "2010s/classification_tasks", "paper_abstract_labels.json")  # 提取后的保存路径

matched_count = extract_valid_papers(large_json_path, valid_papers, output_path)

data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data


Processing JSON: 100%|███████████████████████████████████████████████████████▉| 177G/177G [23:12<00:00, 127MB/s]


2025-02-11 17:33:27 -- Successfully extracted 1527417 paper abstracts


In [12]:
import json

def read_lines(file_path, n: int=2):
    """
    从 Flat JSON 文件中读取前两行并打印解析结果。
    
    Args:
        file_path (str): Flat JSON 文件路径（每行一个 JSON 对象）。
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        for idx, line in enumerate(f):
            if idx < n:  # 只处理前n行
                try:
                    parsed_line = json.loads(line.strip())  # 解析 JSON 对象
                    print(json.dumps(parsed_line, indent=4, ensure_ascii=False))  # 美化输出
                except json.JSONDecodeError as e:
                    print(f"解析失败：{e}, 原始数据：{line.strip()}")
            else:
                break

In [13]:
output_path = os.path.join(get_data_dir(), "2010s/classification_tasks", "paper_abstract_labels.json")
read_lines(output_path, 20)

data_dir:  /home/lyuzhuoqi/projects/clustering/data
{
    "PaperID": 2055207790,
    "abstract": "Matrix metalloproteinase-13 (MMP-13), referred to as collagenase-3, is a proteolytic enzyme that plays a key role in degradation and remodelling of host extracellular matrix proteins. The objective of this study was to characterize the MMP-13 gene in channel catfish, and to determine its pattern of expression in various healthy tissues and during embryogenesis. Since MMP-13 has been shown to have importance in tissue remodelling and some pathological processes, we further studied its involvement in the defense responses of catfish after bacterial infection. The channel catfish MMP-13 cDNA contains an open reading frame of 1416 bp encoding 471 amino acids. Using RT-PCR analysis, MMP-13 was widely expressed in various health tissues. Using quantitative real-time PCR analysis, expression of MMP-13 gene was up-regulated by bacterial infection. During normal embryological development, MMP-13 ex